In [1]:
import pandas as pd

In [2]:
llm_external = pd.read_csv('4epochs_mistral_external_test_predictions.csv')
llm_internal = pd.read_csv('4epochs_mistral_internaltest_predictions.csv')
zer_in = pd.read_csv('mstral_zero_predictions.csv')
zer_ex = pd.read_csv('test_external_zero_predictions.csv')

In [12]:
llm_external.replace('die',0 , inplace=True)
llm_external.replace('survive',1 , inplace=True)

llm_internal.replace('die',0 , inplace=True)
llm_internal.replace('survive',1 , inplace=True)

zer_in.replace('die',0 , inplace=True)
zer_in.replace('survive',1 , inplace=True)

zer_ex.replace('die',0 , inplace=True)
zer_ex.replace('survive',1 , inplace=True)

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

# metrics caculator function 

def calculate_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    cm = confusion_matrix(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted)
    recall = recall_score(y_test, y_predicted)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) != 0 else 0
    f1 = f1_score(y_test, y_predicted)
    
    return accuracy, precision, recall, specificity, f1


def calculate_and_update_model_metrics(y_test, y_predicted,y_ex, y_ex_predicted):
    accuracy, precision, recall, specificity, f1 = calculate_metrics(y_test, y_predicted)
    model = {}
    model["accuracy"] = accuracy
    model["precision"] = precision
    model["recall"] = recall
    model["specificity"] = specificity
    model["f1"] = f1
    model["AUC"] = roc_auc_score(y_test, y_predicted)
    
    accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex = calculate_metrics(y_ex, y_ex_predicted)
    model["accuracy_ex"] = accuracy_ex
    model["precision_ex"] = precision_ex
    model["recall_ex"] = recall_ex
    model["specificity_ex"] = specificity_ex
    model["f1_ex"] = f1_ex
    model["AUC_ex"] = roc_auc_score(y_ex, y_ex_predicted)
    return model


In [27]:
data = calculate_and_update_model_metrics(llm_internal['y_true'].tolist(), llm_internal['y_pred'].tolist(),llm_external['y_true'].tolist(), llm_external['y_pred'].tolist())
df = pd.DataFrame(data , index=[0])

In [28]:
data = calculate_and_update_model_metrics(zer_in['y_true'].tolist(), zer_in['y_pred'].tolist(),zer_ex['y_true'].tolist(), zer_ex['y_pred'].tolist())
df2 = pd.DataFrame(data , index=[0])

In [29]:
c = pd.concat([df , df2] ,axis= 0 )

In [32]:
c.to_excel('fine_tuned-and-zero_shot.xlsx', index=False)